##### Thư viện và Hàm

In [5]:
import pandas as pd
from datetime import datetime, timedelta
import re
import os
import numpy as np

In [6]:
#Đọc thư mục chứa file parquet
def Read_parquet_folder(path, from_date = 0):
    list_file = os.listdir(path)
    new_list = []
    for i in range(len(list_file)):
        if int(list_file[i][-16:-8]) >= from_date:
            new_list.append(list_file[i])
        
    df = pd.DataFrame()
    for file in new_list:
        file_path = os.path.join(path, file)
        df_temp = pd.read_parquet(file_path)
        df = pd.concat([df, df_temp], ignore_index=True)
    return df

In [7]:
def get_latest_date(folder_path: str, pattern: str = r"(\d{8})(?=\.parquet$)") -> str:
    pat = re.compile(pattern, re.IGNORECASE)
    latest_date = None

    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".parquet"):
            m = pat.search(filename)
            if m:
                d = datetime.strptime(m.group(1), "%Y%m%d")
                if latest_date is None or d > latest_date:
                    latest_date = d

    if not latest_date:
        raise ValueError("Không tìm thấy file phù hợp trong thư mục")

    return latest_date.strftime("%Y%m%d")

##### Code xử lí

In [ ]:
latest_date = get_latest_date(r'C:\Users\minhd\Bách hóa XANH\DATA\PARQUET\report-911')
from_date = int(latest_date) - 7
df = Read_parquet_folder(r'C:\Users\minhd\Bách hóa XANH\DATA\PARQUET\report-911', from_date = from_date)
latest_date = pd.to_datetime(latest_date)
df = df[(df['Ngày nhận hàng'] >= latest_date + timedelta(days=-5)) &
        (df['Ngày nhận hàng'] <= latest_date + timedelta(days=3))]

df = df[df['Ngành hàng'] == '1254-Thủy Hải Sản Các Loại']
df = df[df['Mã PO'].notna()]
df = df[['Ngày nhận hàng','Ngành hàng','Nhóm hàng','Tên sản phẩm','Tên siêu thị','Số lượng cần mua','Quy cách mua']]

df['Ngày nhận hàng'] = pd.to_datetime(df['Ngày nhận hàng']).dt.date
df['Mã sản phẩm'] = df['Tên sản phẩm'].str.extract('(^[0-9]{0,13})').astype('int64')
df['Tên sản phẩm'] = df['Tên sản phẩm'].str.extract(r'^[0-9]+-(.*)$')
df['Mã siêu thị'] = df['Tên siêu thị'].str.extract('(^[0-9]{0,5})').astype('int64')

df = df.sort_values(by=['Ngày nhận hàng'], ascending=False)
df = df.drop_duplicates(subset=['Mã siêu thị','Mã sản phẩm'], keep='first')

df = df[['Mã siêu thị','Nhóm hàng','Tên sản phẩm','Số lượng cần mua','Quy cách mua']]

# df['Min chia'] = np.select(
#     [
#         (df['Tên sản phẩm'] == 'CÁ NỤC LÀM SẠCH'),
#         True,
#     ],
#     [
#         3,
#         df['Quy cách mua']
#     ]
# )
df['Min chia'] = df['Quy cách mua']

df['Trạng thái chia hàng'] = np.select(
    [
        df['Số lượng cần mua'] <= df['Min chia'],
        True,
    ],
    [
        'Min',
        'Sức bán'
    ]
)

df = df.rename(columns={'Số lượng cần mua': 'Số mua'})
df = df.sort_values(by=['Mã siêu thị','Trạng thái chia hàng','Số mua','Trạng thái chia hàng'], ascending=[True, True, False, False])
df = df.drop_duplicates(subset=['Mã siêu thị','Tên sản phẩm'], keep='first')

df['Ngày cập nhật'] = datetime.now().date().strftime('%d-%m-%Y')

df_store = pd.read_excel(r"D:\ALL FRESH\KHAI BÁO\Danh sách siêu thị.xlsx", sheet_name='Phân bổ')
df_store.columns = df_store.iloc[0]
df_store = df_store[1:].reset_index(drop=True)

columns_to_select = [0, 1, 8, 10, 11, 12, 13, 14, 16, 19, 20]  # Vị trí cột
df_store = df_store.iloc[:, columns_to_select]
df_store = df_store.rename(columns={'MST': 'Mã siêu thị', 'Tên ST':'Tên siêu thị'})
df_store = df_store[['Mã siêu thị','Tên siêu thị']]

df = pd.merge(df, df_store, on='Mã siêu thị', how='left')

df.to_parquet(r'D:\LINE chatbot\data.parquet', index=False)
df.head()

,Mã siêu thị,Nhóm hàng,Tên sản phẩm,Số mua,Quy cách mua,Min chia,Trạng thái chia hàng,Ngày cập nhật,Tên siêu thị
0,161,3900-Cá nước ngọt,CÁ CHIM NƯỚC NGỌT LÀM SẠCH,2.0,2.0,2.0,Min,30-09-2025,BHX_HCM_BTA - 20 Trương Phước Phan
1,161,3900-Cá nước ngọt,CÁ DIÊU HỒNG LÀM SẠCH,2.0,2.0,2.0,Min,30-09-2025,BHX_HCM_BTA - 20 Trương Phước Phan
2,161,4261-Thủy Sản Chế Biến,CUA ĐỒNG XAY 250GR,2.0,2.0,2.0,Min,30-09-2025,BHX_HCM_BTA - 20 Trương Phước Phan
3,161,3899-Cá biển,CÁ HƯỜNG LÀM SẠCH,2.0,2.0,2.0,Min,30-09-2025,BHX_HCM_BTA - 20 Trương Phước Phan
4,161,4619-CÁ - THỦY HẢI SẢN ĐÓNG GÓI,SỨA ĂN LIỀN XUÂN THANH GÓI 350G,2.0,2.0,2.0,Min,30-09-2025,BHX_HCM_BTA - 20 Trương Phước Phan
